<a href="https://colab.research.google.com/github/John-Macao/Simulacion-JohnMacao/blob/main/prueba_simulacion_eventos_discretos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install simpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import simpy
import random
import matplotlib.pyplot as plt

In [23]:
#Variables
max_vehiculos = 50
num_carriles = 3

tiempo_llegada = 3

tiempo_estaciona = 4
tiempo_turno = 2
tiempo_matricula = 7
tiempo_revision = 15
tiempo_sello = 2
tiempo_salida = 7

tiempo_simulacion = 30

tiempo = {}

class Revision(object):

  def __init__(self, environment, num_carriles, tiempo_estaciona, tiempo_turno, tiempo_matricula, tiempo_revision, tiempo_sello, tiempo_salida):
    self.env=environment
    self.carriles = simpy.Resource(environment, num_carriles)
    self.tiempo_estaciona = tiempo_estaciona
    self.tiempo_turno = tiempo_turno
    self.tiempo_matricula = tiempo_matricula
    self.tiempo_revision = tiempo_revision
    self.tiempo_sello = tiempo_sello
    self.tiempo_salida = tiempo_salida
  
  def estacion_vehiculo(self, vehiculo):
    yield self.env.timeout(random.randint(self.tiempo_estaciona-1, self.tiempo_estaciona+1 ))
    print('Vehiculo estacionado %s ----- : ' %(vehiculo) )
  
  def sacar_turno_vehiculo(self, vehiculo):
    yield self.env.timeout(random.randint(self.tiempo_turno-1, self.tiempo_turno+1 ))
    print('Vehiculo saca turno %s ----- : ' %(vehiculo) )

  def paga_matricula_vehiculo(self, vehiculo):
    yield self.env.timeout(random.randint(self.tiempo_matricula-2, self.tiempo_matricula+3 ))
    print('Vehiculo paga matricula %s ----- : ' %(vehiculo) )

  def sello_vehiculo(self, vehiculo):
    yield self.env.timeout(random.randint(self.tiempo_sello-1, self.tiempo_sello+1 ))
    print('Vehiculo sellado %s ----- : ' %(vehiculo) )
  
  def salida_vehiculo(self, vehiculo):
    yield self.env.timeout(random.randint(self.tiempo_salida-2, self.tiempo_salida+3 ))
    print('Vehiculo sale %s ----- : ' %(vehiculo) )
  
  def revision_vehiculo(self, vehiculo):
    yield self.env.timeout(random.randint(self.tiempo_revision-5,self.tiempo_revision+5)) #cambiar por desviacion estandar
    porcentaje = random.randint(1,100)
    if porcentaje >= 41:
      print('Revision vehiculo aprobado => %s ', vehiculo)
      #pasa a sello
    else:
      print('Revision vehiculo NO PASA => %s ', vehiculo)
      #no pasa a sello
  
  

    


def llegada_vehiculo(env, nombre, revision):
  print('llega vehiculo: %s al tiempo  %.2f.' % (nombre, env.now))

  with revision.carriles.request() as carril:
    yield carril
    print('Entra vehiculo --%s-- a revision hora : %.2f.' %(nombre, env.now))
    yield env.process(revision.estacion_vehiculo(nombre))
    yield env.process(revision.sacar_turno_vehiculo(nombre))
    yield env.process(revision.paga_matricula_vehiculo(nombre))
    yield env.process(revision.revision_vehiculo(nombre))
    
    yield env.process(revision.sello_vehiculo(nombre))
    yield env.process(revision.salida_vehiculo(nombre))





def ejecuta(env, num_carriles, tiempo_llegada, tiempo_estaciona, tiempo_turno, tiempo_matricula, tiempo_sello, tiempo_salida):
  revision = Revision(env, num_carriles, tiempo_estaciona, tiempo_turno, tiempo_matricula, tiempo_revision, tiempo_sello, tiempo_salida)
  
  for i in range(10):
    env.process(llegada_vehiculo(env, 'Vehiculo-%d'%(i+1), revision))

    while True:
      #llega un nuevo vehiculo
      yield env.timeout(tiempo_llegada)
      i+=1
      env.process(llegada_vehiculo(env, 'Vehiculo-%d'%(i+1), revision))


#Simulacion
print('Simulación EMOV')
random.seed()
env = simpy.Environment()
env.process(ejecuta(env, num_carriles, tiempo_llegada, tiempo_estaciona, tiempo_turno, tiempo_matricula, tiempo_sello, tiempo_salida))
env.run(until = tiempo_simulacion )

print(tiempo)






Simulación EMOV
llega vehiculo: Vehiculo-1 al tiempo  0.00.
Entra vehiculo --Vehiculo-1-- a revision hora : 0.00.
llega vehiculo: Vehiculo-2 al tiempo  3.00.
Entra vehiculo --Vehiculo-2-- a revision hora : 3.00.
Vehiculo estacionado Vehiculo-1 ----- : 
llega vehiculo: Vehiculo-3 al tiempo  6.00.
Entra vehiculo --Vehiculo-3-- a revision hora : 6.00.
Vehiculo saca turno Vehiculo-1 ----- : 
Vehiculo estacionado Vehiculo-2 ----- : 
llega vehiculo: Vehiculo-4 al tiempo  9.00.
Vehiculo estacionado Vehiculo-3 ----- : 
Vehiculo saca turno Vehiculo-2 ----- : 
llega vehiculo: Vehiculo-5 al tiempo  12.00.
Vehiculo saca turno Vehiculo-3 ----- : 
Vehiculo paga matricula Vehiculo-2 ----- : 
llega vehiculo: Vehiculo-6 al tiempo  15.00.
Vehiculo paga matricula Vehiculo-1 ----- : 
llega vehiculo: Vehiculo-7 al tiempo  18.00.
Vehiculo paga matricula Vehiculo-3 ----- : 
llega vehiculo: Vehiculo-8 al tiempo  21.00.
llega vehiculo: Vehiculo-9 al tiempo  24.00.
llega vehiculo: Vehiculo-10 al tiempo  27.00.
